# Data Preprocessing
We consider the opening price of the S&P 500 index (^GSPC), and the following are the steps to preprocess the training data:
- Download the data (not shown in this notebook).
- Convert it into a PyTorch tensor.
- Apply a rolling window.

where the last two steps are included in the ``get_stock_price`` function. Then we have to
- sample *time* indices,
- apply augmentation(s), and 
- transform the data into signatures.

In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
import os
import pandas as pd
import signatory
from lib.datasets import get_stock_price,sample_indices,train_test_split
from lib.aug import augment_path_and_compute_signatures,apply_augmentations,parse_augmentations

In [3]:
path = os.path.join("datasets", "stock", "^GSPC_1d.csv")
df = pd.read_csv(path)
print(f'Original data: {os.path.basename(path)}, shape {df.shape}')

Original data: ^GSPC_1d.csv, shape (1251, 7)


In [4]:
data_config = {
    "ticker" : "^GSPC",
    "interval" : "1d",
    "column" : 1,  
    "window_size" : 20,
    "dir" : "datasets",
    "subdir" : "stock"
}

In [5]:
sig_config = {
    "augmentations": [
        {"name": "AddTime"},
        {"name": "LeadLag"},
    ],
    "device" : "cuda:0",
    "depth" : 5,
}

Convert the data into PyTorch tensor.

In [6]:
tensor_data = get_stock_price(data_config)

Rolled data for training, shape torch.Size([1232, 20, 1])


Separate training data and testing data.

In [7]:
x_real_train, x_real_test = train_test_split(tensor_data, train_test_ratio=0.8, device='CUDA')

Apply augmentations. In our experiments, we do not apply augmentations and use the original signatures.

In [8]:
if sig_config["augmentations"] is not None:
    sig_config["augmentations"] = parse_augmentations(sig_config.get('augmentations'))

In [9]:
y = x_real_train
print("Before augmentation shape:",y.shape)
if sig_config["augmentations"] is not None:
    # Print the tensor shape after each augmentation
    y_aug = apply_augmentations(y,sig_config["augmentations"])
print("After augmentation shape:",y_aug.shape)

Before augmentation shape: torch.Size([985, 20, 1])
torch.Size([985, 20, 2])
torch.Size([985, 39, 4])
After augmentation shape: torch.Size([985, 39, 4])


Sample time indices. The amount of the sampling is the batch size.

In [10]:
batch_size = 128    
data_size = y_aug.shape[0]
time_indices = sample_indices(data_size,batch_size,'cpu')

Sample signatures according to ``time_indices``.

In [11]:
sample = y_aug[time_indices]
print(sample.shape)

torch.Size([128, 39, 4])


Calculate signature of degree 4.

In [12]:
sig_sample = signatory.signature(sample,4)
print(sig_sample.shape)

torch.Size([128, 340])


We will use three test metrics in our experiments: 
- JS divergence 
- L2 norm
- signature MMD.

We verify that the distance two identical signatures is zero.
For the L2 norm and JSD, we will take the mean over the batch dimension for convenience.

In [13]:
def kl_divergence(P, Q, eps=1e-10):
    """
    Compute KL divergence between two sets of distributions.
    
    Args:
        P (torch.Tensor): Tensor of shape [batch, dim], the first distribution.
        Q (torch.Tensor): Tensor of shape [batch, dim], the second distribution.
        eps (float): Small constant to avoid numerical instability (default: 1e-10).
    
    Returns:
        kl (torch.Tensor): KL divergence for each batch, shape [batch].
        kl_scalar (torch.Tensor): Scalar KL divergence (mean over batches).
    """
    # Add epsilon to avoid log(0) or division by zero
    P = P + eps
    Q = Q + eps
    
    # Normalize over the dim dimension to ensure they are probability distributions
    P = P / P.sum(dim=-1, keepdim=True)
    Q = Q / Q.sum(dim=-1, keepdim=True)
    
    # Compute KL divergence: P * log(P / Q)
    kl = P * torch.log(P / Q)  # Element-wise operation
    kl = kl.sum(dim=-1)        # Sum over dim, resulting in shape [batch]
    
    # Reduce to a scalar (mean over batches)
    kl_scalar = kl.mean()
    
    return kl, kl_scalar

In [14]:
def mmd_loss(S_X: torch.Tensor, S_Y: torch.Tensor) -> torch.Tensor:
    '''
    S_X: torch.Tensor of shape (batch_X, dim) - precomputed signatures of paths
    S_Y: torch.Tensor of shape (batch_Y, dim) - precomputed signatures of paths
    '''
    # Compute Gram matrices using inner product of signatures
    K_XX = S_X @ S_X.T
    K_YY = S_Y @ S_Y.T
    K_XY = S_X @ S_Y.T

    # Get the number of samples in X and Y
    n = S_X.shape[0]
    m = S_Y.shape[0]

    # Compute the unbiased MMD statistic
    # Sum off-diagonal elements of K_XX and K_YY
    sum_XX = torch.sum(K_XX) - torch.trace(K_XX)
    sum_YY = torch.sum(K_YY) - torch.trace(K_YY)
    sum_XY = torch.sum(K_XY)

    # Compute MMD^2
    mmd_squared = (sum_XX / (n * (n - 1)) + sum_YY / (m * (m - 1)) - 2 * sum_XY / (n * m))

    return mmd_squared

In [17]:
sample1, sample2 = sig_sample, sig_sample
# Mean of L2 norm over the batch dimension
mse = torch.nn.functional.mse_loss(sample1,sample2) 
# Mean of KL divergence over the batch dimension
_, KLD1 = kl_divergence(sample1,sample2)
_, KLD2 = kl_divergence(sample2,sample1)
js_mean_batch = (KLD1+KLD2)/2
mmd = mmd_loss(sample1,sample2)
print("Mean of L2 norm: {}".format(mse.item()))
print("Mean of JSD: {}".format(js_mean_batch.item()))
print("MMD loss: {}".format(mmd.item()))

Mean of L2 norm: 0.0
Mean of JSD: 0.0
MMD loss: -2.474303858629673e+18
